In [17]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer

In [18]:
sw=stopwords.words('english')
lemma=WordNetLemmatizer()

In [3]:
pos_rev=pd.read_csv('pos.txt',sep='\n',encoding='latin-1',header=None)

In [4]:
pos_rev['mood']=1

In [5]:
pos_rev.rename(columns={0:'review'},inplace=True)

In [6]:
pos_rev

,review,mood
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
5326,both exuberantly romantic and serenely melanch...,1
5327,mazel tov to a film about a family's joyous li...,1
5328,standing in the shadows of motown is the best ...,1
5329,it's nice to see piscopo again after all these...,1


In [7]:
neg_rev=pd.read_csv('negative.txt',sep='\n',encoding='latin-1',header=None)
neg_rev['mood']=0
neg_rev.rename(columns={0:'review'},inplace=True)

In [8]:
neg_rev

,review,mood
0,"simplistic , silly and tedious.",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0
...,...,...
5326,a terrible movie that some people will neverth...,0
5327,there are many definitions of 'time waster' bu...,0
5328,"as it stands , crocodile hunter has the hurrie...",0
5329,the thing looks like a made-for-home-video qui...,0


In [9]:
# lower case
pos_rev['review']=pos_rev['review'].apply(lambda x:x.lower())

In [11]:
# remove punctuations
pos_rev['review']=pos_rev['review'].apply(lambda x:' '.join([i for i in nltk.word_tokenize(x) if i not in string.punctuation]))

In [14]:
# remove stopwords
pos_rev['review']=pos_rev['review'].apply(lambda x:' '.join([i for i in nltk.word_tokenize(x) if i not in sw]))

In [20]:
# lemmatize
pos_rev['review']=pos_rev['review'].apply(lambda x:lemma.lemmatize(x))

In [22]:
# lower case
neg_rev['review']=neg_rev['review'].apply(lambda x:x.lower())
# remove punctuations
neg_rev['review']=neg_rev['review'].apply(lambda x:' '.join([i for i in nltk.word_tokenize(x) if i not in string.punctuation]))
# remove stopwords
neg_rev['review']=neg_rev['review'].apply(lambda x:' '.join([i for i in nltk.word_tokenize(x) if i not in sw]))
# lemmatize
neg_rev['review']=neg_rev['review'].apply(lambda x:lemma.lemmatize(x))

In [24]:
# connecting the dATA SETS
common_rev=pd.concat([pos_rev,neg_rev],axis=0).reset_index()
common_rev

,index,review,mood
0,0,rock destined 21st century 's new `` conan `` ...,1
1,1,gorgeously elaborate continuation `` lord ring...,1
2,2,effective too-tepid biopic,1
3,3,sometimes like go movies fun wasabi good place...,1
4,4,emerges something rare issue movie 's honest k...,1
...,...,...,...
10657,5326,terrible movie people nevertheless find moving,0
10658,5327,many definitions 'time waster movie must surel...,0
10659,5328,stands crocodile hunter hurried badly cobbled ...,0
10660,5329,thing looks like made-for-home-video quickie,0


In [26]:
x_train,x_test,y_train,y_test=train_test_split(common_rev['review'].values,common_rev['mood'].values,test_size=0.2,random_state=42)

In [29]:
train_data = pd.DataFrame({'review' : x_train , 'mood':y_train})
test_data = pd.DataFrame({'review' : x_test , 'mood':y_test})

In [31]:
vector=TfidfVectorizer()
train_vectors=vector.fit_transform(train_data['review'])
test_vectors=vector.transform(test_data['review'])

In [33]:
from sklearn.svm import SVC

In [36]:
# model training
classifier=SVC()
classifier.fit(train_vectors,train_data['mood'])

SVC()

In [38]:
# Predictions
pred=classifier.predict(test_vectors)
pred

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [41]:
from sklearn.metrics import classification_report
classification_report(test_data['mood'],pred,output_dict=True)

{'0': {'precision': 0.7504604051565378,
  'recall': 0.7609710550887021,
  'f1-score': 0.7556791840519239,
  'support': 1071},
 '1': {'precision': 0.7554918815663801,
  'recall': 0.7448210922787194,
  'f1-score': 0.7501185395922237,
  'support': 1062},
 'accuracy': 0.7529301453352086,
 'macro avg': {'precision': 0.752976143361459,
  'recall': 0.7528960736837107,
  'f1-score': 0.7528988618220738,
  'support': 2133},
 'weighted avg': {'precision': 0.7529655284323242,
  'recall': 0.7529301453352086,
  'f1-score': 0.7529105931394993,
  'support': 2133}}